# Constrained and Guided Output Generation

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

#Helper Function

In [10]:
def display_output(output):
    """Displays the genrated output"""
    print("Model Output:")
    print("-" * 40)
    print(output)
    print("-" * 40)
    print()

#Setting up Constraints

In [11]:
from langchain_core.prompts import PromptTemplate

constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""Create a product description for {product} targeted at {target_audience}.
    Use a {tone} tone and keep it under {word_limit} words.
    The description should include:
    1. A catchy headline
    2. Three key features
    3. A call to action
    
    Product Description:
    """
)
input_variables = {
    "product": "smart water bottle",
    "target_audience": "health-conscious millennials",
    "tone": "casual and friendly",
    "word_limit": "75"
}

chain = constrained_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
**Hydrate Like a Pro: Your New BFF**

Stay on top of your hydration game with our smart water bottle! Here's what sets us apart:

Track your water intake and set goals to stay on track
Monitor your temperature and get reminders to drink up
Sync with your phone to track your progress and earn rewards

Ready to level up your hydration game? Get your smart water bottle now and start sipping your way to a healthier you!
----------------------------------------



#Implementing rule based generation

In [12]:
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

input_variables = {
    "job_title": "GenAI Engineer",
    "company": "TechInnovate Solutions",
    "location": "Banglore, India",
    "experience": "3+"
}

chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
Here is the job posting for a GenAI Engineer position at TechInnovate Solutions in Bangalore, India:

COMPANY: TechInnovate Solutions is a cutting-edge technology firm that specializes in developing innovative artificial intelligence solutions for various industries. We strive to stay at the forefront of AI research and development, pushing the boundaries of what is possible with machine learning and deep learning technologies.

RESPONSIBILITIES:
- Design and develop cutting-edge GenAI models that drive business value for our clients
- Collaborate with cross-functional teams to identify and prioritize AI opportunities that align with business goals
- Develop and maintain high-quality, scalable, and reusable AI components and frameworks
- Conduct experiments and analyze results to improve model performance and accuracy
- Stay up-to-date with the latest advancements in AI research and apply this knowledge to drive innovation at TechI

In [14]:
from langchain.output_parsers import RegexParser
import re

# Define a regex parser for structured output
regex_parser = RegexParser(
    regex=r"COMPANY:\s*([\s\S]*?)\n\s*RESPONSIBILITIES:\s*([\s\S]*?)\n\s*QUALIFICATIONS:\s*([\s\S]*?)\n\s*EEO:\s*([\s\S]*)",
    output_keys=["company_description", "responsibilities", "qualifications", "eeo_statement"]
)
# This regex pattern captures the company description, responsibilities, qualifications, and EEO statement from the output text.

# Create a new prompt template that includes the parser instructions
parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output EXACTLY as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r'\n\s*', '\n', value.strip())
    return output

# Generate the parsed output
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse and clean the output
parsed_output = regex_parser.parse(raw_output)
cleaned_output = clean_output(parsed_output)

# Display the parsed output
print("Parsed Output:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

Parsed Output:
COMPANY_DESCRIPTION:
TechInnovate Solutions is a cutting-edge technology company that specializes in developing innovative AI-powered solutions for various industries. We are committed to pushing the boundaries of artificial intelligence and machine learning to create a better future.

RESPONSIBILITIES:
- Design and develop advanced AI models and algorithms for various applications
- Collaborate with cross-functional teams to integrate AI solutions with existing systems
- Develop and maintain high-quality AI-powered software applications
- Conduct data analysis and visualization to identify trends and patterns
- Stay up-to-date with the latest advancements in AI and machine learning and apply this knowledge to improve our solutions

QUALIFICATIONS:
- 3+ years of experience in AI engineering or a related field
- Master's or Ph.D. degree in Computer Science, Artificial Intelligence, or a related field
- Strong programming skills in languages such as Python, Java, or C++
- 

In [15]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:
    1. The review should have a {rating}-star rating (out of 5)
    2. Include exactly {pros} pros and {cons} cons
    3. Use between 2 and 3 sentences for each pro and con
    4. The entire review should be under {word_limit} words
    5. End with a one-sentence recommendation
    
    Format the review as follows:
    Rating: [X] out of 5 stars
    
    Pros:
    1. [Pro 1]
    2. [Pro 2]
    ...
    
    Cons:
    1. [Con 1]
    2. [Con 2]
    ...
    
    Recommendation: [One-sentence recommendation]
    """
)

# Generate the constrained review
input_variables = {
    "product": "Smartphone X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200"
}

chain = review_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
Rating: 4 out of 5 stars

Pros:
1. The Smartphone X boasts an impressive display with vibrant colors and crisp details, making it a joy to watch videos and play games on.
2. The phone's camera is also a major highlight, with excellent low-light performance and features like optical zoom and portrait mode.
3. The device's battery life is surprisingly long-lasting, easily lasting a full day with moderate to heavy use.

Cons:
1. The phone's processor can feel a bit sluggish at times, particularly when switching between demanding apps or performing complex tasks.
2. The phone's design, while sleek, feels a bit too light and may not appeal to those who prefer a more substantial device.

Recommendation: Overall, Smartphone X is a solid choice for anyone looking for a reliable and feature-packed device, but those seeking a premium feel and top-notch performance may want to consider other options.
----------------------------------------

